# Session Objects
The Session object allows you to persist certain parameters across requests. It also persists cookies across all requests made from the Session instance, and will use urllib3's connection pooling. So if you're making several requests to the same host, the underlying TCP connection will be reused, which can result in a significant performance increase (see HTTP persistent connection).

A Session object has all the methods of the main Requests API.

Let's persist some cookies across requests:

In [30]:
import requests
import json
import pprint

In [2]:
s = requests.Session()

s.get('http://httpbin.org/cookies/set/sessioncookie/123456789')
r = s.get('http://httpbin.org/cookies')

print(r.text)

{
  "cookies": {
    "sessioncookie": "123456789"
  }
}



Sessions can also be used to provide default data to the request methods. This is done by providing data to the properties on a Session object:

In [3]:
s = requests.Session()
s.auth = ('user', 'pass')
s.headers.update({'x-test': 'true'})

# both 'x-test' and 'x-test2' are sent
s.get('http://httpbin.org/headers', headers={'x-test2': 'true'})

<Response [200]>

Any dictionaries that you pass to a request method will be merged with the session-level values that are set. The method-level parameters override session parameters.

Note, however, that method-level parameters will not be persisted across requests, even if using a session. This example will only send the cookies with the first request, but not the second:

In [4]:
s = requests.Session()

r = s.get('http://httpbin.org/cookies', cookies={'from-my': 'browser'})
print(r.text)
# '{"cookies": {"from-my": "browser"}}'

r = s.get('http://httpbin.org/cookies')
print(r.text)

{
  "cookies": {
    "from-my": "browser"
  }
}

{
  "cookies": {}
}



# Request and Response Objects


Whenever a call is made to `requests.get()` and friends, you are doing two major things. First, you are constructing a Request object which will be sent off to a server to request or query some resource. Second, a Response object is generated once Requests gets a response back from the server. The Response object contains all of the information returned by the server and also contains the Request object you created originally. Here is a simple request to get some very important information from Wikipedia's servers:

In [5]:
 r = requests.get('http://en.wikipedia.org/wiki/Monty_Python')

If we want to access the headers the server sent back to us, we do this:



In [7]:
r.headers

{'Date': 'Fri, 30 Mar 2018 00:47:13 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '73503', 'Connection': 'keep-alive', 'Server': 'mw1262.eqiad.wmnet', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'X-Powered-By': 'HHVM/3.18.6-dev', 'Link': '</static/images/project-logos/enwiki.png>;rel=preload;as=image;media=not all and (min-resolution: 1.5dppx),</static/images/project-logos/enwiki-1.5x.png>;rel=preload;as=image;media=(min-resolution: 1.5dppx) and (max-resolution: 1.999999dppx),</static/images/project-logos/enwiki-2x.png>;rel=preload;as=image;media=(min-resolution: 2dppx)', 'X-UA-Compatible': 'IE=Edge', 'Content-language': 'en', 'P3P': 'CP="This is not a P3P policy! See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'X-Content-Type-Options': 'nosniff', 'Last-Modified': 'Wed, 28 Mar 2018 13:25:58 GMT', 'Backend-Timing': 'D=142366 t=1522243699331958', 'Content-Encoding': 'gzip', 'X-Varnish': '419632814 356747693, 202075172 203209078, 

In [8]:
type(r)

requests.models.Response

However, if we want to get the headers we sent the server, we simply access the request, and then the request's headers:

In [9]:
r.request.headers

{'User-Agent': 'python-requests/2.18.4', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Cookie': 'WMF-Last-Access=30-Mar-2018; WMF-Last-Access-Global=30-Mar-2018'}

In [11]:
r = requests.get('https://api.github.com/repos/requests/requests/issues/482')
r.status_code

200

In [14]:
issue = json.loads(r.text)

print(issue[u'title'])

Feature any http verb in docs


In [31]:
pprint.pprint(issue)

{'assignee': None,
 'assignees': [],
 'author_association': 'OWNER',
 'body': '',
 'closed_at': '2012-05-29T23:50:05Z',
 'closed_by': {'avatar_url': 'https://avatars2.githubusercontent.com/u/119893?v=4',
               'events_url': 'https://api.github.com/users/kennethreitz/events{/privacy}',
               'followers_url': 'https://api.github.com/users/kennethreitz/followers',
               'following_url': 'https://api.github.com/users/kennethreitz/following{/other_user}',
               'gists_url': 'https://api.github.com/users/kennethreitz/gists{/gist_id}',
               'gravatar_id': '',
               'html_url': 'https://github.com/kennethreitz',
               'id': 119893,
               'login': 'kennethreitz',
               'organizations_url': 'https://api.github.com/users/kennethreitz/orgs',
               'received_events_url': 'https://api.github.com/users/kennethreitz/received_events',
               'repos_url': 'https://api.github.com/users/kennethreitz/repos',


In [33]:
r.url

'https://api.github.com/repos/requests/requests/issues/482/comments'

In [15]:
print(issue[u'comments'])


10


In [17]:
type(issue)

dict

In [19]:
issue['comments']

10

In [20]:
r = requests.get(r.url + u'/comments')
r.status_code

200

In [22]:
comments = r.json()
print(comments[0].keys())

dict_keys(['url', 'html_url', 'issue_url', 'id', 'user', 'created_at', 'updated_at', 'author_association', 'body'])


In [38]:
pprint.pprint(comments[0])

{'author_association': 'OWNER',
 'body': "As @kennethreitz knows, I'm preparing to use requests in a different "
         'project, and was testing out the GitHub API. The following is '
         'something I did from the python command line interpreter:\n'
         '\n'
         '``` python\n'
         'import requests\n'
         "a = ('sigmavirus24', 'not_my_real_password')\n"
         'url = '
         "'https://api.github.com/repos/sigmavirus24/issues.py/issues/comments/4524692'  "
         "# This no longer exists, you'll see why\n"
         'data = u\'{"body": "Amend comment."}\'\n'
         'r = requests.patch(url, data, auth=a)\n'
         '# Checked what GitHub returned (response code 200)\n'
         "r = requests.request('DELETE', url, auth=a)\n"
         '# GitHub responded with code 204 -- successful deletion of a comment '
         'on an issue.\n'
         '```\n',
 'created_at': '2012-03-15T17:42:58Z',
 'html_url': 'https://github.com/requests/requests/issues/482#issue

In [26]:
 print(comments[2][u'body'])

Probably in the "advanced" section



In [27]:
print(comments[2][u'user'][u'login'])

kennethreitz
